In [2]:
using Pkg; Pkg.activate("../../../FermiCG/")
using FermiCG
using PyCall
using Plots
using LinearAlgebra
using Printf
using JLD2

  Activating project at `~/code/FermiCG`
┌ Info: Precompiling FermiCG [855c3b3a-d9bb-4919-8b00-0b3cfbdffb6f]
└ @ Base loading.jl:1423
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1423


In [10]:
#molecule = "
#He 0.00000000 0.00000000 0.00000000
#He 1.41421356 0.00000000 0.00000000
#He 0.00000000 1.41421356 0.00000000
#He 1.41421356 1.41421356 0.00000000
#He 0.70710678 0.70710678 1.00000000
#He 0.70710678 0.70710678 -1.00000000
#"
molecule = "
He 0.00000000 0.00000000 0.00000000
He 2.12132035 0.00000000 0.00000000
He 0.00000000 2.12132035 0.00000000
He 2.12132034 2.12132034 0.00000000
He 1.06066017 1.06066017 1.50000000
He 1.06066017 1.06066017 -1.50000000
"

atoms = []
for (li,line) in enumerate(split(rstrip(lstrip(molecule)), "\n"))
    l = split(line)
    push!(atoms, Atom(li, l[1], parse.(Float64,l[2:4])))
end

basis = "aug-cc-pvdz"

# Create FermiCG.Molecule type
mol     = Molecule(0, 1, atoms,basis);

pyscf = pyimport("pyscf")

n_steps = 35
step_size = .05

pymol_init = pyscf.gto.Mole(atom=molecule,
                            symmetry = false, spin =0,charge=0,
                            basis = basis)
pymol_init.build()

PyObject <pyscf.gto.mole.Mole object at 0x7fb0f02dd910>

In [11]:
io = open("traj.xyz", "w");
energies_ground = []
energies_t1 = []
energies_t2 = []
energies_t3 = []
energies_t4 = []
energies_t5 = []
energies_t6 = []

energies_s1 = []
energies_s2 = []
energies_s3 = []
energies_s4 = []
energies_s5 = []
energies_s6 = []

#energies_t7 = []
#energies_t8 = []
#energies_t9 = []
#energies_t10 = []
#energies_t11 = []
#energies_t12 = []
pt2_energies = []

lo = pyimport("pyscf.lo.orth")
tools = pyimport("pyscf.tools")
fcidump = pyimport("pyscf.tools.fcidump");

In [ ]:
for R in 1:n_steps

    pymol = deepcopy(pymol_init)
    scale = 1+R*step_size
    
    #move to smaller geometry
    xyz = @sprintf("%5i\n\n", length(mol.atoms))
    tmp = []
    for a in mol.atoms
        push!(tmp, ["He", (a.xyz[1]/scale, a.xyz[2]/scale, a.xyz[3]/scale)])
        xyz = xyz * @sprintf("%6s %24.16f %24.16f %24.16f \n", a.symbol, a.xyz[1]/scale, a.xyz[2]/scale, a.xyz[3]/scale)
    end
    
    #move to larger geometry
    #for a in mol.atoms
    #    push!(tmp, ["He", (a.xyz[1]*scale, a.xyz[2]*scale, a.xyz[3]*scale)])
    #    xyz = xyz * @sprintf("%6s %24.16f %24.16f %24.16f \n", a.symbol, a.xyz[1]*scale, a.xyz[2]*scale, a.xyz[3]*scale)
    #end
    #pymol.atom = tmp
    #pymol.build()


    println(xyz)
    write(io, xyz);


    #println(pymol.format_atom(1))

    #     mol_R = Molecule(0, 1, [a[0]pymol.atom, pymol.basis)


    mf = pyscf.scf.RHF(pymol).run()
    
    s = mf.get_ovlp(pymol)

    lo_ao = lo.lowdin(s)
    println("size of Lowdin ortho AO's:", size(lo_ao))

    #write fci dump file from the modified mo coefficients
    tools.fcidump.from_mo(pymol, "fcidump.he06_oct", lo_ao)

    #Can just read in pyscf dump file for integrals (once you have already run an scf calculation)
    ctx = fcidump.read("fcidump.he06_oct");
    h = ctx["H1"];
    g = ctx["H2"];
    ecore = ctx["ECORE"];
    g = pyscf.ao2mo.restore("1", g, size(h,2))

    #This one below was not working. Error: setfield! immutable struct of type InCoreInts cannot be changed
    ints = InCoreInts(ecore,h,g);

    #Run cmf
    #Define clusters and intial Fock space for inital CMF calc for 14 orbs each He
    #clusters_in = [(1:14),(15:28), (29:42), (43:56), (57:70), (71:84), (85:98)]

    #Define clusters and intial Fock space for inital CMF calc for 9 orbs each He
    clusters_in = [(1:9),(10:18), (19:27), (28:36), (37:45), (46:54)]

    #Define clusters and intial Fock space for inital CMF calc for 5 orbs each He
    #clusters_in = [(1:5),(6:10), (11:15), (16:20), (21:25), (26:30)]
    init_fspace = [(1,1),(1,1),(1,1),(1,1),(1,1),(1,1)]
    rdm1 = zeros(size(ints.h1))
    na=6
    nb=6

    #Define clusters now using FermiCG code
    clusters = [Cluster(i,collect(clusters_in[i])) for i = 1:length(clusters_in)]
    display(clusters)

    #@save "before_cmf.jld2" ints clusters init_fspace

    print(size(ints.h1))
    rdm1 = zeros(size(ints.h1))
    
    #do a CMF calculation to optimize cluster orbitals

    e_cmf, U, Da, Db = FermiCG.cmf_oo(ints, clusters, init_fspace, rdm1, rdm1, max_iter_oo=11*0.0500, verbose=0, gconv=1e-6, method="cg", sequential=true);
    #FermiCG.pyscf_write_molden(mol,lo_ao*U, filename="cmf.molden");
    #error("stop here")

    #rotate the integrals by the cmf calculation
    ints = FermiCG.orbital_rotation(ints, U);
    max_roots = 40

    #Build Cluster Basis (delta n is here)
    cluster_bases = FermiCG.compute_cluster_eigenbasis(ints, clusters, delta_elec=2, verbose=1, max_roots=max_roots, init_fspace=init_fspace, rdm1a=Da, rdm1b=Db);

    #@save "scan_after_cmf.jld2" ints Da Db e_cmf cluster_bases clusters init_fspace

    #Build Clustered Operator
    cluster_ham = FermiCG.extract_ClusteredTerms(ints, clusters);
    
    #Build Cluster Operators
    cluster_ops = FermiCG.compute_cluster_ops(cluster_bases, ints);
    FermiCG.add_cmf_operators!(cluster_ops, cluster_bases, ints, Da, Db);
    
    #Need to find reference state 
    ref_fock = FermiCG.FockConfig(init_fspace)
    nroots = 13
    #ci_vector = FermiCG.TPSCIstate(clusters, ref_fock, R=nroots)
    ci_vector = FermiCG.TPSCIstate(clusters, FermiCG.FockConfig(init_fspace), R=nroots);
    #ci_vector = FermiCG.ClusteredState(clusters, ref_fock, R=nroots);
    #Need to find the automated way to define these other excited configs away from ref state, example is to large
    #to do by hand
    #probably something to do with building p spaces and q spaces
    
    ci_vector[ref_fock][ClusterConfig([1,1,1,1,1,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([2,1,1,1,1,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,2,1,1,1,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,1,2,1,1,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,1,1,2,1,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,1,1,1,2,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,1,1,1,1,2])] = zeros(Float64,nroots)
    
    ci_vector[ref_fock][ClusterConfig([3,1,1,1,1,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,3,1,1,1,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,1,3,1,1,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,1,1,3,1,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,1,1,1,3,1])] = zeros(Float64,nroots)
    ci_vector[ref_fock][ClusterConfig([1,1,1,1,1,3])] = zeros(Float64,nroots)
    
    #ci_vector[ref_fock][ClusterConfig([4,1,1,1,1,1])] = zeros(Float64,nroots)
    #ci_vector[ref_fock][ClusterConfig([1,4,1,1,1,1])] = zeros(Float64,nroots)
    #ci_vector[ref_fock][ClusterConfig([1,1,4,1,1,1])] = zeros(Float64,nroots)
    #ci_vector[ref_fock][ClusterConfig([1,1,1,4,1,1])] = zeros(Float64,nroots)
    #ci_vector[ref_fock][ClusterConfig([1,1,1,1,4,1])] = zeros(Float64,nroots)
    #ci_vector[ref_fock][ClusterConfig([1,1,1,1,1,4])] = zeros(Float64,nroots)

    FermiCG.eye!(ci_vector)
    display(ci_vector)

    #for thresh_cipsi in thresh_list
    e0, v0 = FermiCG.tpsci_ci(ci_vector, cluster_ops, cluster_ham,
                              thresh_cipsi=1e-3, # Threshold for adding to P-space
                              #thresh_cipsi=thresh_cipsi, # Threshold for adding to P-space
                              thresh_foi=1e-2,    # Threshold for keeping terms when defining FOIS
                              thresh_asci=1e-1,     # Threshold of P-space configs to search from
                              max_iter=10);

    @time e2 = FermiCG.compute_pt2_energy(v0, cluster_ops, cluster_ham, thresh_foi=1e-8)
    #name = "eq_tpsci_results"*string(0.001)*".jld2"
    #@save name e0 e2 v0 ecore
    
    println()
    println("	*======TPSCI results======*")
    @printf("TCI Thresh: %8.6f  Dim:%8d\n",1e-2,size(v0)[1])
    println()
    @printf("TCI %5s %12s %12s\n", "Root", "E(0)", "E(2)") 
    for r in 1:nroots
        @printf("TCI %5s %12.8f %12.8f\n",r, e0[r] + ecore, e0[r] + e2[r] + ecore)
        display(v0,thresh=1e-4,root=r)
    end
    #end
    
    push!(energies_ground, e0[1]+ecore)
    push!(energies_t1, e0[2]+ecore)
    push!(energies_t2, e0[3]+ecore)
    push!(energies_t3, e0[4]+ecore)
    push!(energies_t4, e0[5]+ecore)
    push!(energies_t5, e0[6]+ecore)
    push!(energies_t6, e0[7]+ecore)
    
    push!(energies_s1, e0[8]+ecore)
    push!(energies_s2, e0[9]+ecore)
    push!(energies_s3, e0[10]+ecore)
    push!(energies_s4, e0[11]+ecore)
    push!(energies_s5, e0[12]+ecore)
    push!(energies_s6, e0[13]+ecore)

    #push!(energies_t7, e0[14]+ecore)
    #push!(energies_t8, e0[15]+ecore)
    #push!(energies_t9, e0[16]+ecore)
    #push!(energies_t10, e0[17]+ecore)
    #push!(energies_t11, e0[18]+ecore)
    #push!(energies_t12, e0[19]+ecore)

    push!(pt2_energies, e2)
end

    6

    He       0.0000000000000000       0.0000000000000000       0.0000000000000000 
    He       2.2273863674999999       0.0000000000000000       0.0000000000000000 
    He       0.0000000000000000       2.2273863674999999       0.0000000000000000 
    He       2.2273863570000003       2.2273863570000003       0.0000000000000000 
    He       1.1136931785000002       1.1136931785000002       1.5750000000000002 
    He       1.1136931785000002       1.1136931785000002      -1.5750000000000002 

Parsing fcidump.he06_oct
converged SCF energy = -17.1253607068562
size of Lowdin ortho AO's:(54, 54)


6-element Vector{Cluster}:
 Cluster(1, [1, 2, 3, 4, 5, 6, 7, 8, 9])
 Cluster(2, [10, 11, 12, 13, 14, 15, 16, 17, 18])
 Cluster(3, [19, 20, 21, 22, 23, 24, 25, 26, 27])
 Cluster(4, [28, 29, 30, 31, 32, 33, 34, 35, 36])
 Cluster(5, [37, 38, 39, 40, 41, 42, 43, 44, 45])
 Cluster(6, [46, 47, 48, 49, 50, 51, 52, 53, 54])

(54, 54) ooCMF Iter:    1 Total= -17.022413982514 Active= -30.442054225000 G=     1.93e+00
 ooCMF Iter:    2 Total= -17.240116594374 Active= -30.659756836860 G=     1.14e+00
 ooCMF Iter:    3 Total= -17.316882407147 Active= -30.736522649634 G=     1.71e-01
 ooCMF Iter:    4 Total= -17.318817089005 Active= -30.738457331492 G=     7.84e-02
 ooCMF Iter:    5 Total= -17.319273212503 Active= -30.738913454990 G=     4.98e-02
 ooCMF Iter:    6 Total= -17.319450923204 Active= -30.739091165691 G=     3.54e-02
 ooCMF Iter:    7 Total= -17.319951561940 Active= -30.739591804426 G=     5.07e-02
 ooCMF Iter:    8 Total= -17.320828783195 Active= -30.740469025682 G=     1.23e-01
 ooCMF Iter:    9 Total= -17.323238990433 Active= -30.742879232919 G=     1.28e-01
 ooCMF Iter:   10 Total= -17.325507718810 Active= -30.745147961297 G=     5.56e-02
 ooCMF Iter:   11 Total= -17.325711217002 Active= -30.745351459489 G=     5.45e-02
 ooCMF Iter:   12 Total= -17.325827202076 Active= -30.745467444563 G=     2.12e

 ooCMF Iter:  100 Total= -17.326154636512 Active= -30.745794878998 G=     8.65e-04
 ooCMF Iter:  101 Total= -17.326154705540 Active= -30.745794948027 G=     5.36e-04
 ooCMF Iter:  102 Total= -17.326154722779 Active= -30.745794965265 G=     2.78e-04
 ooCMF Iter:  103 Total= -17.326154731387 Active= -30.745794973873 G=     7.29e-04
 ooCMF Iter:  104 Total= -17.326154767630 Active= -30.745795010116 G=     5.96e-04
 ooCMF Iter:  105 Total= -17.326154819645 Active= -30.745795062131 G=     5.19e-04
 ooCMF Iter:  106 Total= -17.326154869358 Active= -30.745795111844 G=     9.20e-04
 ooCMF Iter:  107 Total= -17.326155051467 Active= -30.745795293954 G=     1.37e-03
 ooCMF Iter:  108 Total= -17.326155251940 Active= -30.745795494427 G=     1.89e-03
 ooCMF Iter:  109 Total= -17.326155533907 Active= -30.745795776394 G=     9.91e-04
 ooCMF Iter:  110 Total= -17.326155605706 Active= -30.745795848193 G=     1.05e-03
 ooCMF Iter:  111 Total= -17.326155652127 Active= -30.745795894613 G=     4.44e-04
 ooC

 ooCMF Iter:  199 Total= -17.326158303488 Active= -30.745798545975 G=     5.72e-04
 ooCMF Iter:  200 Total= -17.326158322618 Active= -30.745798565105 G=     5.02e-04
 ooCMF Iter:  201 Total= -17.326158339698 Active= -30.745798582184 G=     2.96e-04


...

 FCIProblem:: #Orbs = 9   #α = 2  #β = 0  Dimension: 36       
   State    1 Energy:  -2.11129310  10.14142190
   State    2 Energy:  -1.88224757  10.37046743
   State    3 Energy:  -1.88224757  10.37046743
   State    4 Energy:  -1.77185722  10.48085778
   State    5 Energy:  -1.05563343  11.19708158
   State    6 Energy:  -0.57165074  11.68106427
   State    7 Energy:  -0.57165071  11.68106429
   State    8 Energy:  -0.47135061  11.78136440
   State    9 Energy:  -0.39717115  11.85554385
   State   10 Energy:  -0.29503309  11.95768191
   State   11 Energy:  -0.29503307  11.95768193
   State   12 Energy:   0.16893055  12.42164556
   State   13 Energy:   0.22432841  12.47704341
   State   14 Energy:   0.22432842  12.47704342
   State   15 Energy:   0.33531290  12.58802791
   State   16 Energy:   0.36411932  12.61683432
   State   17 Energy:   0.36411933  12.61683434
   State   18 Energy:   0.45941783  12.71213284
   State   19 Energy:   1.32699295  13.57970796
   State   20 Energy:   1

 FCIProblem:: #Orbs = 9   #α = 0  #β = 2  Dimension: 36       
   State    1 Energy:  -2.11129310  10.14142190
   State    2 Energy:  -1.88224757  10.37046743
   State    3 Energy:  -1.88224757  10.37046743
   State    4 Energy:  -1.77185722  10.48085778
   State    5 Energy:  -1.05563343  11.19708158
   State    6 Energy:  -0.57165074  11.68106427
   State    7 Energy:  -0.57165071  11.68106429
   State    8 Energy:  -0.47135061  11.78136440
   State    9 Energy:  -0.39717115  11.85554385
   State   10 Energy:  -0.29503309  11.95768191
   State   11 Energy:  -0.29503307  11.95768193
   State   12 Energy:   0.16893055  12.42164556
   State   13 Energy:   0.22432841  12.47704341
   State   14 Energy:   0.22432842  12.47704342
   State   15 Energy:   0.33531290  12.58802791
   State   16 Energy:   0.36411932  12.61683432
   State   17 Energy:   0.36411933  12.61683434
   State   18 Energy:   0.45941783  12.71213284
   State   19 Energy:   1.32699295  13.57970796
   State   20 Energy:   1

 FCIProblem:: #Orbs = 9   #α = 2  #β = 0  Dimension: 36       
   State    1 Energy:  -2.11129310  10.14142190
   State    2 Energy:  -1.88224757  10.37046743
   State    3 Energy:  -1.88224757  10.37046743
   State    4 Energy:  -1.77185722  10.48085778
   State    5 Energy:  -1.05563343  11.19708158
   State    6 Energy:  -0.57165074  11.68106427
   State    7 Energy:  -0.57165071  11.68106429
   State    8 Energy:  -0.47135061  11.78136440
   State    9 Energy:  -0.39717115  11.85554385
   State   10 Energy:  -0.29503309  11.95768191
   State   11 Energy:  -0.29503307  11.95768193
   State   12 Energy:   0.16893055  12.42164556
   State   13 Energy:   0.22432841  12.47704341
   State   14 Energy:   0.22432842  12.47704342
   State   15 Energy:   0.33531290  12.58802791
   State   16 Energy:   0.36411932  12.61683432
   State   17 Energy:   0.36411933  12.61683434
   State   18 Energy:   0.45941783  12.71213284
   State   19 Energy:   1.32699295  13.57970796
   State   20 Energy:   1

Excessive output truncated after 524351 bytes.

   State    1 Energy:  -2.66526489   9.58745012
   State    2 Energy:  -2.28041185   9.97230316
   State    3 Energy:  -2.28041182   9.97230319
   State    4 Energy:  -2.15979828  10.09291672


In [ ]:
@save "scan_energies_triplet.jld2" energies_ground energies_t1 energies_t2 energies_t3 energies_t4 energies_t5 energies_t6
@save "scan_energies_singlet.jld2" energies_s1 energies_s2 energies_s3 energies_s4 energies_s5 energies_s6
#@save "scan_energies_second_triple.jld2" energies_t7 energies_t8 energies_t9 energies_t10 energies_t11 energies_t12
@save "pt2_energies.jld2" pt2_energies